<a href="https://colab.research.google.com/github/manyamrupa123/OCR-of-Bank-Statements/blob/main/OCR_of_Bank_Statements.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 !pip install cloudinary

In [ ]:
import cloudinary
import cloudinary.uploader

cloudinary.config(
    cloud_name='djgwiaqq3',
    api_key='578563758427597',
    api_secret='pmNXK8Uv0SJ_--XA6v5W_hM9tAE'
)

In [ ]:
!pip install gradio
!pip install pytesseract
!pip install requests
!pip install pandas
!pip install matplotlib

In [ ]:
import cloudinary
import cloudinary.api
import requests
import os
import gradio as gr
import pytesseract
from PIL import Image
import re
import tempfile
import matplotlib.pyplot as plt
import pandas as pd

# Cloudinary Configuration
cloudinary.config(
    cloud_name='djgwiaqq3',
    api_key='578563758427597',
    api_secret='pmNXK8Uv0SJ_--XA6v5W_hM9tAE'
)

# Predefined field names and default values for document types
FIELD_NAMES = {
    "Payslip": ["salary"],
    "Profit and loss statements": ["net salary"],
    "cheques": ["rupees"]
}

# Function to fetch available folders in Cloudinary
def fetch_cloudinary_folders():
    try:
        response = cloudinary.api.root_folders()
        folders = [folder['name'] for folder in response.get('folders', [])]
        return folders
    except Exception as e:
        print(f"Error fetching folders: {e}")
        return []

# Function to retrieve images from a Cloudinary folder
def fetch_images_from_cloudinary(folder_name, num_images):
    image_paths = []
    next_cursor = None
    retrieved_count = 0

    while retrieved_count < num_images:
        try:
            response = cloudinary.api.resources(
                type="upload", prefix=folder_name,
                max_results=min(100, num_images - retrieved_count),
                next_cursor=next_cursor
            )
            images = response.get('resources', [])
            next_cursor = response.get('next_cursor')

            for image in images:
                if retrieved_count >= num_images:
                    break
                image_url = image['secure_url']
                img_data = requests.get(image_url).content
                with tempfile.NamedTemporaryFile(suffix=".jpg", delete=False) as tmpfile:
                    tmpfile.write(img_data)
                    image_paths.append(tmpfile.name)
                retrieved_count += 1

            if not next_cursor:
                break
        except Exception as e:
            return f"Error retrieving images: {str(e)}", []
    return "Images retrieved successfully!", image_paths

# Function to extract text from an image using OCR
def extract_text_from_image(image):
    text = pytesseract.image_to_string(image, config='--psm 6')
    return text

# Function to clean extracted values
def clean_values(values):
    cleaned_values = []
    for value in values:
        try:
            cleaned_values.append(float(value))
        except (ValueError, TypeError):
            pass
    return cleaned_values

# Function to extract, store in table format, visualize, and generate JSON
def extract_and_visualize_from_cloudinary(folder_name, num_images, chart_type, document_type):
    # Fetch images from the specified Cloudinary folder
    status, image_paths = fetch_images_from_cloudinary(folder_name, int(num_images))
    if "Error" in status:
        return status, None, None, None, None, None

    extracted_data = {field: [] for field in FIELD_NAMES.get(document_type, [])}
    for img_path in image_paths:
        try:
            image = Image.open(img_path)
            text = extract_text_from_image(image)
            for field in FIELD_NAMES.get(document_type, []):
                extracted_data[field].append(text)  # Simplified text extraction for now
        except Exception as e:
            return f"Error processing image: {str(e)}", None, None, None, None, None

    cleaned_data = {}
    summed_data = {}
    for field, values in extracted_data.items():
        cleaned_values = clean_values(values)
        if cleaned_values:  # Exclude empty or zero-like data
            cleaned_data[field] = cleaned_values
            summed_data[field] = sum(cleaned_values)

    # Add default values for missing fields
    default_values = DEFAULT_VALUES[document_type]
    summed_data.update(default_values)

    # Create a JSON representation without zero-like fields
    json_data = {key: value for key, value in summed_data.items() if value}

    # Create DataFrame for visualization
    min_max_data = {"Field": list(default_values.keys()), "Value": list(default_values.values())}
    min_max_df = pd.DataFrame(min_max_data)

    # Visualization
    fig, ax = plt.subplots(figsize=(10, 6))
    colors = plt.cm.tab20.colors

    if chart_type == "Bar Chart":
        ax.bar(json_data.keys(), json_data.values(), color=colors[:len(json_data)])
        ax.set_title(f"{document_type} Data Visualization (Bar Chart)")
        ax.set_ylabel('Total Value')
        ax.set_xlabel('Fields')
        plt.xticks(rotation=45, ha="right")
    elif chart_type == "Pie Chart":
        ax.pie(
            json_data.values(),
            labels=json_data.keys(),
            autopct='%1.1f%%',
            startangle=140,
            colors=colors[:len(json_data)]
        )
        ax.set_title(f"{document_type} Data Visualization (Pie Chart)")

    plt.tight_layout()

    # Save the chart as a temporary image file
    with tempfile.NamedTemporaryFile(suffix=".png", delete=False) as tmpfile:
        chart_path = tmpfile.name
        plt.savefig(chart_path)

    plt.close()

    # Save DataFrame as CSV
    with tempfile.NamedTemporaryFile(suffix=".csv", delete=False) as csvfile:
        csv_path = csvfile.name
        min_max_df.to_csv(csv_path, index=False)

    return min_max_df.to_html(index=False), chart_path, json_data, csv_path

# Gradio interface
def create_gradio_interface():
    cloudinary_folders = fetch_cloudinary_folders()

    with gr.Blocks() as interface:
        gr.Markdown("""
            <div style="text-align: center; font-size: 24px; font-weight: bold;">
                 Cloudinary Image OCR and Data Visualization
            </div>
            <div style="text-align: center; font-size: 16px; color: gray;">
                Select your options below, and visualize extracted data efficiently.
            </div>
        """)

        # Input Section
        gr.Markdown("### Input Options")
        with gr.Row():
            folder_name = gr.Dropdown(cloudinary_folders, label="Select Cloudinary Folder", value=cloudinary_folders[0] if cloudinary_folders else None)
            document_type = gr.Dropdown(["Payslip", "Profit and loss statements", "cheques"], label="Document Type", value="Payslip")
        with gr.Row():
            num_images = gr.Slider(minimum=1, maximum=10, step=1, label="Number of Images to Retrieve", value=10)
            chart_type = gr.Radio(["Bar Chart", "Pie Chart"], label="Chart Type", value="Bar Chart")

        # Submit Button
        submit_btn = gr.Button("Fetch, Extract, and Visualize")

        # Output Section
        gr.Markdown("Highest and Lowest Values Table")
        min_max_table = gr.HTML()
        gr.Markdown("Visualization")
        output_chart = gr.Image()
        gr.Markdown("JSON Output")
        json_output = gr.JSON(label="Extracted Data JSON")
        gr.Markdown("Download Extracted Data as CSV")
        download_csv = gr.File(label="Download CSV")

        # Button Click Action
        submit_btn.click(
            extract_and_visualize_from_cloudinary,
            inputs=[folder_name, num_images, chart_type, document_type],
            outputs=[min_max_table, output_chart, json_output, download_csv]
        )

    interface.launch(share=True)

# Run the Gradio interface
if __name__ == "__main__":
    create_gradio_interface()
